In [25]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import Input, Dense, Concatenate
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

###Load data

In [2]:
housing = datasets.fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing["data"],housing["target"])
x_train, x_val, y_train, y_val = train_test_split(x_train_full,y_train_full)

In [3]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [5]:
3e-3

0.003

###In order to fin the best parameters using RandomSearchCv from Sklearn, we should wrap a function that build the model into a sklearn regressor

In [18]:
from tqdm import tqdm

In [21]:
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape = [8]):
  model = keras.models.Sequential()
  options = {"input_shape": input_shape}

  for layer in tqdm(range(n_hidden)):
    model.add(Dense(n_neurons, activation= "relu", **options))
    options = {}
  
  model.add(Dense(1, **options))
  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(optimizer= optimizer , loss = "mse")

  return model

In [23]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [26]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal

In [28]:
param_distribs = {
    "n_hidden": [0,1,2,3],
    "n_neurons": np.arange(1,100),
    "learning_rate" : reciprocal(3e-4,3e-2)
}

Note thath randomized search uses Kfold, so we don't need actually the validation set. But here the val set is helpful to use the EarlyStopping.

In [30]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter = 10, cv = 3)
rnd_search_cv.fit(x_train, y_train, epochs = 100,
                  validation_data = (x_val, y_val),
                  callbacks = [keras.callbacks.EarlyStopping(patience=10)])

100%|██████████| 2/2 [00:00<00:00, 88.60it/s]


Epoch 1/100
242/242 [==============================] - 1s 3ms/step - loss: 0.9887 - val_loss: 0.7407
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 1.0457 - val_loss: 0.5047
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4991 - val_loss: 0.6056
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.7824 - val_loss: 0.5571
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4677 - val_loss: 0.4298
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4271 - val_loss: 0.4216
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4145 - val_loss: 0.4066
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4047 - val_loss: 0.4041
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3967 - val_loss: 0.4009
Epoch 10/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3895 - val_lo

100%|██████████| 2/2 [00:00<00:00, 68.26it/s]


Epoch 1/100
242/242 [==============================] - 1s 3ms/step - loss: 0.8577 - val_loss: 0.8365
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 1.3292 - val_loss: 0.6212
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.7280 - val_loss: 0.4960
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5106 - val_loss: 0.4418
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4464 - val_loss: 0.4317
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4286 - val_loss: 0.4127
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4158 - val_loss: 0.4009
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4052 - val_loss: 0.3925
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3955 - val_loss: 0.3919
Epoch 10/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3892 - val_lo

100%|██████████| 2/2 [00:00<00:00, 89.00it/s]


...

0it [00:00, ?it/s]


Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 2.1274 - val_loss: 0.6710
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6126 - val_loss: 0.5977
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5709 - val_loss: 0.5778
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5499 - val_loss: 0.5709
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5366 - val_loss: 0.5687
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5281 - val_loss: 0.5741
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5217 - val_loss: 0.5818
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5182 - val_loss: 0.5932
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5153 - val_loss: 0.6045
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5131 - val_lo

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-3.11269323e-01             nan -4.35262670e-01 -3.26442818e-01
 -4.70452497e-01 -4.37666704e-01 -4.96492356e-01 -4.95538396e+03
             nan -3.28740278e+06]
  category=UserWarning,
100%|██████████| 2/2 [00:00<00:00, 91.47it/s]


Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 0.7717 - val_loss: 0.5864
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.7061 - val_loss: 0.5918
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.7805 - val_loss: 0.4466
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4454 - val_loss: 0.4130
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4166 - val_loss: 0.3970
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4021 - val_loss: 0.3901
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3909 - val_loss: 0.3808
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3813 - val_loss: 0.3730
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3737 - val_loss: 0.3746
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3673 - val_lo

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7ff037527590>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff032ba13d0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [31]:
rnd_search_cv.best_params_

{'learning_rate': 0.008082769211802867, 'n_hidden': 2, 'n_neurons': 63}